In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## 使用 BigQuery DataFrames 與生成式 AI 進行程式碼生成

<table align="left">

  <td>
    <a href="https://colab.sandbox.google.com/github/doggy8088/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bq_dataframes_llm_code_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中執行
    </a>
  </td>
  <td>
    <a href="https://github.com/doggy8088/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bq_dataframes_llm_code_generation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/language/use-cases/applying-llms-to-data/bq_dataframes_llm_code_generation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中開啟
    </a>
  </td>                                                                                               
</table>


| | |
|-|-|
|作者 | [Ashley Xu](https://github.com/ashleyxuu) |


**_注意_** : 已在下列環境測試筆記本：

* Python 版本 = 3.10


## 概觀

使用此筆記本逐步了解一個範例使用案例，其中使用 BigQuery DataFrames 及其與 Vertex AI 上的生成式 AI 支援整合來產生範例程式碼。

進一步瞭解 [BigQuery DataFrames](https://cloud.google.com/python/docs/reference/bigframes/latest)。


### 目標

本教學課程中，你將建立一個包含範例程式碼以呼叫指定 API 程式的 CSV 檔案。

步驟包括：

- 在 BigQuery DataFrames 中定義 LLM 模型，特別是 PaLM API 的 [「文字-bison」模型](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text)(採用 `bigframes.ml.llm`)。
- 透過從 Cloud Storage 讀取資料來建立 DataFrame。
- 操作 DataFrame 中的資料，以建置 LLM 提示。
- 使用 `predict` 方法將 DataFrame 提示傳送至 LLM 模型。
- 建立並使用自訂函式，以轉換 LLM 模型回應提供的輸出。
- 將轉換後的結果 DataFrame 匯出為 CSV 檔案。


### 資料集

本教學使用一個列出各種 Pandas DataFrame 和 Series API 名稱的資料集。


### 成本

本教學課程使用 Google Cloud 的計費元件：

* BigQuery
* Vertex AI 上的生成式 AI 支援
* Cloud Functions

進一步了解 [BigQuery 計算定價](https://cloud.google.com/bigquery/pricing#analysis_pricing_models)、[Vertex AI 上的生成式 AI 支援定價](https://cloud.google.com/vertex-ai/pricing#generative_ai_models) 以及 [Cloud Functions 定價](https://cloud.google.com/functions/pricing)，
並使用 [定價計算器](https://cloud.google.com/products/calculator/)，根據預計用量來產生成本估算。


## 安裝

安裝以下套件，執行這個筆記本需要這些套件：


In [ ]:
!pip install bigframes --upgrade --quiet

## 開始前

完成本節中的任務以設定你的環境。


### 設定 Google Cloud 專案

**無論你使用哪種類型的筆記本環境，都必須執行下列步驟。** 

1. [選擇或建立 Google Cloud 專案](https://console.cloud.google.com/cloud-resource-manager)。初次建立帳戶時，你可獲得 300 美元的額度來支付運算/儲存成本。

2. [確保已為專案啟用帳單](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [按一下這裡](https://console.cloud.google.com/flows/enableapi?apiid=bigquery.googleapis.com,bigqueryconnection.googleapis.com,cloudfunctions.googleapis.com,run.googleapis.com,artifactregistry.googleapis.com,cloudbuild.googleapis.com,cloudresourcemanager.googleapis.com) 以啟用下列 API：

  * BigQuery API
  * BigQuery Connection API
  * Cloud Functions API
  * Cloud Run API
  * Artifact Registry API
  * Cloud Build API
  * Cloud Resource Manager API
  * Vertex AI API

4. 如果你是在本機執行此筆記本，請安裝 [Cloud SDK](https://cloud.google.com/sdk)。


#### 設定你的專案 ID

如果你不知道你的專案 ID，請嘗試以下：
* 執行 `gcloud config list`。
* 執行 `gcloud projects list`。
* 查看支援頁： [找出專案 ID](https://support.google.com/googleapi/answer/7014113)。


In [ ]:
PROJECT_ID = ""  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 設定區域

你也可以變更 BigQuery 使用的 `REGION` 變數。進一步了解 [BigQuery 區域](https://cloud.google.com/bigquery/docs/locations#supported_locations)。


In [ ]:
REGION = "US"  # @param {type: "string"}

### 驗證 Google Cloud 帳戶

根據你的 Jupyter 環境，你可能必須手動驗證。依照相關指示操作。


**Vertex AI Workbench** 

保持不動，你已得到身分驗證。


**本地 JupyterLab 範例** 

取消註解並執行以下Cell：


In [ ]:
# ! gcloud auth login

**Colab** 

取消註釋並執行以下Cell：


In [ ]:
# from google.colab import auth
# auth.authenticate_user()

### 匯入函式庫


In [ ]:
import bigframes.pandas as bf
from google.cloud import bigquery_connection_v1 as bq_connection

### 設定 BigQuery DataFrames 選項


In [ ]:
bf.options.bigquery.project = PROJECT_ID
bf.options.bigquery.location = REGION

如果你想重設建立的 DataFrame 或 Series 物件的位置，執行 `bf.reset_session()` 重設工作階段。然後，你可以重複使用 `bf.options.bigquery.location` 指定另一個位置。


# 定義 LLM 模型

BigQuery DataFrame 提供與 PaLM API 的 [`text-bison` 模型](https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/text) 的整合，透過 Vertex AI。

本部分將逐步說明在筆記本中使用模型所需的步驟。


## 建立 BigQuery Cloud 資源連接

你需要建立 [Cloud 資源連接](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection) 以讓 BigQuery 資料架構能與 Vertex AI 服務互動。


In [ ]:
CONN_NAME = "bqdf-llm"

client = bq_connection.ConnectionServiceClient()
new_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}"
exists_conn_parent = f"projects/{PROJECT_ID}/locations/{REGION}/connections/{CONN_NAME}"
cloud_resource_properties = bq_connection.CloudResourceProperties({})

try:
    request = client.get_connection(
        request=bq_connection.GetConnectionRequest(name=exists_conn_parent)
    )
    CONN_SERVICE_ACCOUNT = f"serviceAccount:{request.cloud_resource.service_account_id}"
except Exception:
    connection = bq_connection.types.Connection(
        {"friendly_name": CONN_NAME, "cloud_resource": cloud_resource_properties}
    )
    request = bq_connection.CreateConnectionRequest(
        {
            "parent": new_conn_parent,
            "connection_id": CONN_NAME,
            "connection": connection,
        }
    )
    response = client.create_connection(request)
    CONN_SERVICE_ACCOUNT = (
        f"serviceAccount:{response.cloud_resource.service_account_id}"
    )
print(CONN_SERVICE_ACCOUNT)

## 對服務帳戶設定權限

資源連線服務帳戶需要特定專案層級的權限：
 - `roles/aiplatform.user` 及 `roles/bigquery.connectionUser`：這些角色是需要連線去使用 Vertex AI 中的 LLM 模型建立模型定義 ([文件](https://cloud.google.com/bigquery/docs/generate-text#give_the_service_account_access))。
 - `roles/run.invoker`：此角色是需要連線存取備份自訂／遠端函式的 Cloud Run 服務之唯讀存取 ([文件](https://cloud.google.com/bigquery/docs/remote-functions#grant_permission_on_function))。

執行以下 `gcloud` 命令設定這些權限：


In [ ]:
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/bigquery.connectionUser'
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/aiplatform.user'
!gcloud projects add-iam-policy-binding {PROJECT_ID} --condition=None --no-user-output-enabled --member={CONN_SERVICE_ACCOUNT} --role='roles/run.invoker'

## 定義模型

使用 bigframes.ml.llm 定義模型:


In [ ]:
from bigframes.ml.llm import PaLM2TextGenerator

session = bf.get_global_session()
connection = f"{PROJECT_ID}.{REGION}.{CONN_NAME}"
model = PaLM2TextGenerator(session=session, connection_name=connection)

# 將 Cloud Storage 中的資料讀取至 BigQuery DataFrames

你可以透過閱讀以下任一位置的資料來建立 BigQuery DataFrames DataFrame：

* 本地資料檔案
* 儲存在 BigQuery 資料表的資料
* 儲存在 Cloud Storage 的資料檔案
* 記憶中的 pandas DataFrame

在本教學課程中，你將透過閱讀儲存在 Cloud Storage 中的兩個 CSV 檔案來建立 BigQuery DataFrames DataFrames，一個包含 DataFrame API 名稱清單，另一個包含 Series API 名稱清單。


In [ ]:
df_api = bf.read_csv("gs://cloud-samples-data/vertex-ai/bigframe/df.csv")
series_api = bf.read_csv("gs://cloud-samples-data/vertex-ai/bigframe/series.csv")

看看每個檔案的數列資料：


In [ ]:
df_api.head(2)

In [ ]:
series_api.head(2)

# 使用 LLM 模型產生程式碼

準備提示，並將其發送到 LLM 模型進行預測。


## 在 BigQuery DataFrames 中使用 Prompt 設計

專門設計 LLM 提示的開發領域快速成長，如需瞭解更多資訊，請參閱 [此文件](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/introduction-prompt-design)。

在這個教學課程中，你使用一個簡單的提示，請 LLM 模型提供上個步驟 DataFrames 中每個 API 方法 (或列) 的範例程式碼。輸出為新的 DataFrames `df_prompt` 和 `series_prompt`，其中包含提示文字。


In [ ]:
df_prompt_prefix = "Generate Pandas sample code for DataFrame."
series_prompt_prefix = "Generate Pandas sample code for Series."

df_prompt = df_prompt_prefix + df_api["API"]
series_prompt = series_prompt_prefix + series_api["API"]

df_prompt.head(2)

## 使用 LLM 範例進行預測

將包含完整提示文字的 BigQuery DataFrames DataFrame 作為 `predixt` 方法的輸入。`predixt` 方法會呼叫 LLM 範例，並將其產生的文字輸出傳回兩個新的 BigQuery DataFrames DataFrames，`df_pred` 和 `series_pred`。

注意：預測可能需要花費幾分鐘時間執行。


In [ ]:
df_pred = model.predict(df_prompt.to_frame(), max_output_tokens=1024)
series_pred = model.predict(series_prompt.to_frame(), max_output_tokens=1024)

一旦預測處理完畢，請檢視 LLM 的範例輸出，其提供 DataFrame 資料集所列 API 名稱的程式碼範例。


In [ ]:
print(df_pred["ml_generate_text_llm_result"].iloc[0])

# 使用遠端函式操控 LLM 輸出

LLM 提供的輸出經常包含超出程式碼範例本身的額外文字。使用 BigQuery DataFrames，你可以部署處理並轉換此輸出的自訂 Python 函式。


執行下方的Cell會建立一個自訂函式，你可以使用這項函式以兩種方式處理 LLM 輸出的資料：
1. 從 LLM 文字輸出中移除程式碼區塊。
2. 將 `import pandas as pd` 取代為 `import bigframes.pandas as bf`，讓產生的程式碼區塊可配合 BigQuery Cell使用。


In [ ]:
@bf.remote_function([str], str, bigquery_connection=CONN_NAME)
def extract_code(text: str):
    try:
        res = text[text.find("\n") + 1 : text.find("```", 3)]
        res = res.replace("import pandas as pd", "import bigframes.pandas as bf")
        if "import bigframes.pandas as bf" not in res:
            res = "import bigframes.pandas as bf\n" + res
        return res
    except:
        return ""

自定義函式部署為 [雲端函式](https://cloud.google.com/functions/docs/)，然後與 BigQuery 整合為 [遠端函式](https://cloud.google.com/bigquery/docs/remote-functions)。儲存兩個函式的名稱，以便你可以在這個筆記本的結尾清理它們。


In [ ]:
CLOUD_FUNCTION_NAME = format(extract_code.bigframes_cloud_function)
print("Cloud Function Name " + CLOUD_FUNCTION_NAME)
REMOTE_FUNCTION_NAME = format(extract_code.bigframes_remote_function)
print("Remote Function Name " + REMOTE_FUNCTION_NAME)

將客製化函式套用於每一 筆 LLM 輸出 DataFrame，以取得處理過的結果：


In [ ]:
df_code = df_pred.assign(
    code=df_pred["ml_generate_text_llm_result"].apply(extract_code)
)
series_code = series_pred.assign(
    code=series_pred["ml_generate_text_llm_result"].apply(extract_code)
)

你可以透過檢視第一行資料來看出差異：


In [ ]:
print(df_code["code"].iloc[0])

# 將結果儲存至雲端儲存空間

BigQuery DataFrames 可讓你將 BigQuery DataFrames DataFrame 作為 CSV 檔案儲存到雲端儲存空間，以利後續使用。現在就使用處理過的 LLM 輸出資料試試看。


建立一個有唯一名稱的新 Cloud Storage 儲存空間：


In [ ]:
import uuid

BUCKET_ID = "code-samples-" + str(uuid.uuid1())

!gsutil mb gs://{BUCKET_ID}

使用 `to_csv` 將每個 BigQuery DataFrame 資料框作為 CSV 檔案寫入 Cloud Storage 儲存空間:


In [ ]:
df_code[["code"]].to_csv(f"gs://{BUCKET_ID}/df_code*.csv")
series_code[["code"]].to_csv(f"gs://{BUCKET_ID}/series_code*.csv")

你可以前往 Cloud Storage 儲存空間瀏覽器下載這兩個檔案並檢視。

執行下列Cell，然後連結至你的 Cloud Storage 儲存空間瀏覽器：


In [ ]:
print(f"https://console.developers.google.com/storage/browser/{BUCKET_ID}/")

# 總結和後續步驟

你已使用 BigQuery DataFrames 與 LLM 模型 (`bigframes.ml.llm`) 的整合產生程式碼範例，並透過在 BigQuery DataFrames 中建立和使用自訂函式來轉換 LLM 輸出。

進一步了解 [文件](https://cloud.google.com/python/docs/reference/bigframes/latest) 中的 BigQuery DataFrames，並在 [GitHub 儲存庫](https://github.com/googleapis/python-bigquery-dataframes/tree/main/notebooks) 中找出更多範例筆記本。


## 清理

如需清理此專案中使用的所有 Google Cloud 資源，你可以 [刪除專案](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) 用於教學課程。

否則，你可以取消註解剩餘的Cell，然後執行它們，以刪除你在此教學課程中建立的個別資源：


In [ ]:
# # Delete the BigQuery Connection
# from google.cloud import bigquery_connection_v1 as bq_connection
# client = bq_connection.ConnectionServiceClient()
# CONNECTION_ID = f"projects/{PROJECT_ID}/locations/{REGION}/connections/{CONN_NAME}"
# client.delete_connection(name=CONNECTION_ID)
# print(f"Deleted connection '{CONNECTION_ID}'.")

In [ ]:
# # Delete the Cloud Function
# ! gcloud functions delete {CLOUD_FUNCTION_NAME} --quiet
# # Delete the Remote Function
# REMOTE_FUNCTION_NAME = REMOTE_FUNCTION_NAME.replace(PROJECT_ID + ".", "")
# ! bq rm --routine --force=true {REMOTE_FUNCTION_NAME}

In [ ]:
# # Delete the Google Cloud Storage bucket and files
# ! gsutil rm -r gs://{BUCKET_ID}
# print(f"Deleted bucket '{BUCKET_ID}'.")